#                            Plotting 2D Harmonic Oscillator Eigenfunctions in 3 dimensions


Solving the 1D Schrödinger Equation with Numerov’s Algorithm

The Schrödinger equation describes the energy and time-evolution of a particle or system of particles, and is one of the fundamental building blocks of modern physics. In it’s general form, the (time-independent) Schrödinger equation looks like this:1

$\frac{-\hbar^2}{2m} \frac{\partial^2}{\partial x^2}\psi(x) + V(x) \psi(x) = E\psi(x)$

There are relatively few situations in which the Schrödinger equation can be solved analytically, and numerical methods and approximations are one way around that analytical limitation. To demonstrate how this is possible and how a numerical solution works, what better way than to solve a system which can be solved analytically and comparing the results.

In solving the Schrödinger equation, we will start with one of the simplest interesting quantum mechanical systems, the quantum mechanical harmonic oscillator.2 Let’s first define our quantum harmonic oscillator. The general form of the Schrödinger equation for a one-dimensional harmonic oscillator reads thus:

$\frac{-\hbar^2}{2m} \frac{\partial^2}{\partial z^2}\psi(z) + V(z) \psi(z) = E\psi(z)$

We will make use of the Numerov algorithm which is particularly suited to solving second order differential equations of the form y′′(x)+k(x)y(x)=0.

The algorithm:

$(1+\frac{1}{12}h^2k_{n+1})y_{n+1} = 2(1−\frac{5}{12}h^2k_{n})y_{n}−(1+\frac{1}{12}h^2k_{n−1})y_{n−1}+O(h^6)  $

As you can see, it provides 6th order accuracy which is pretty impressive for such a simple algorithm. In the above equation, h
is the step size between each iteration, and n is the index of iteration; y and k

relate to those in the formula in the paragraph above.

Thus we need to manipulate (1)
into a (dimensionless) form which the Numerov algorithm can solve: using a substitution E=εℏω and z=xℏmω−−−√ we can rearrange (1)

into the form:

$ ψ′′(x)+(2ε–x^2)ψ(x)=0 $

Now the Schrödinger equation is in the correct form, we can simply plug it into the Numerov algorithm and see what comes out.
Finding the Eigenvalues Numerically

To determine the eigenvalues, the program scans a range of energies, denoted by the Greek letter ε
in the above equations, and tests for when the tail of the graph changes from +∞ to −∞ , or vice versa. When that happens, the tail must have crossed zero, and therefore it must have stepped over a solution.3 The program then goes backwards and so on with increased resolution, honing in until it finds all of the solutions we want.

To convert this from 1D to a 2D <b>circularly symmetrical oscillatory potential</b>, we use a little bit of intuition:

We know our algorithm works for a 1D coordinate system. Since our potential is circularly symmetric, we can expect our wavefunctions to be circularly symmetrical as well. Of course, quantum physics is not that simple. Since we are now working with 2 dimensions, the wavefunctions are defined by not one but two quantum numbers - a radial quantum number <i><b>n</b></i> and an angular quantum number <i><b>l</b></i>. The Numerov algorithm only works for the case where the angular quantum number is zero. The energy eigen numbers are dependent on the radial number, more specifically $E_{n} = (n + 1)$.

Setting this number to zero, we can find the eigenfunctions for each radial quantum number <i><b>n</b></i>. It should be emphasized that the radial quantum number cannot be an odd integer when the angular number is zero. The radial eigenstates are thus all even.


In [ ]:
#Program that plots the eigen functions of a 2D simple harmonic oscillator, with angular quantum number set to zero.
import math
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np


In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')

N=60000 # Number of iterations of the Numerov Algorithm
h = 0.0001 # Step size in the algorithm
h2 = pow(h,2) 


def main():

  epsilon = getEigenState() #Prompts the user for the energy eigenstate to be plotted

  x_out, z_out = NumerovAlg(N, h2, epsilon) 

  p = np.linspace(0, 2*np.pi, 100) 

  # print (x_out)

  R, P = np.meshgrid(x_out, p) 
  #Creates 2D arrays R and P that hold the polar coordinates r and $\theta$ respectively. 

  X, Y = R*np.cos(P), R*np.sin(P) 

  Z = np.array([z_out for i in range(len(P))])
  # Z = np.outer(z_out, z_out)
  print(Z)
  print(X.shape,Y.shape, Z.shape)

  surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                         linewidth=0, antialiased=False)

  # Customize the z axis.
  ax.text2D(0.05, 0.95, "Eigenfunctions with angular quantum number zero for radial quantum number " + str(int(epsilon - 0.5)), transform=ax.transAxes)
  ax.zaxis.set_major_locator(LinearLocator(10))
  ax.set_xlabel('X axis')
  ax.set_ylabel('Y axis')
  ax.set_zlabel('$\Psi$')
  # ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

  # Add a color bar which maps values to colors.
  fig.colorbar(surf, shrink=0.5, aspect=5)

  plt.show()

In [ ]:
# Returns list x_out that is the radial axis, and z_out that is the value of the wavefunction on each axis
def NumerovAlg(N, h2, epsilon):
  z = 0.0
  k = 0.0
  x = -1*(N-2)*h

  k_minus_2 = epsilon + x-2*h # k_0
  k_minus_1 = epsilon + x-h # k_1
  a = 0.1
  z_minus_2 = 0 # z_0
  z_minus_1 = a # z_1

  x_out = []
  z_out = []

  n=-1*N+2
  while n < N-2:
    n+=1
    x += h;
    k = 2*epsilon - pow(x, 2)
    b = h2/12
    z = ( 2*(1-5*b*k_minus_1) * z_minus_1 - (1+b*k_minus_2) * z_minus_2 ) / (1 + b * k)

    # Save for plotting. Add new values of x and z to the lists x_out and z_out
    if (n % 10 == 0):
      x_out.append(x)
      z_out.append(z)

    # Shift for next iteration
    z_minus_2 = z_minus_1
    z_minus_1 = z
    k_minus_2 = k_minus_1
    k_minus_1 = k

  return x_out, z_out #The x_out corresponds to the radial axis since the potential well is radially symmetrical

In [ ]:
def getEigenState():
  epsilon = 0
  while True:
    inp = input("Enter the radial quantum number of the eigenstate you want to plot(even integers only): ")
    
    if inp.isdigit() and int(inp) % 2 == 0:
      epsilon = (0.5 + float(inp))
      break
    else:
      print("That's not a positive integer!")

  return epsilon

In [ ]:
main()

### References
 Numerov Algorithm: http://mtdevans.com/2013/07/solving-the-schrodinger-equation-with-numerovs-algorithm/
 
 Complete two dimensional solutions for 2D harmonic oscillator eigenfunctions: http://ramen.physics.und.edu/~yloh/TEACHING/PHYS536/N3-LandauLevelsFigures.nb.pdf